# Homework 10: Cleaning data with Regular Expressions

Time to use regular expressions!

# Hints and notes

### Opening files in subdirectories

Notice that this notebook might be **homework/**, but!!! the csvs and text files might be in **homework/scraped/** or **/homework/scraped/minutes_pdfs** or **/homework/pdfs/**. To open a file in a subdirectory, instead of having the filename be `"file.csv"` you'll just use `"some/subfolder/file.csv"`

### Opening text files

This will open up a file, read it in and show you the first 500 characters.

```python
contents = open("your-filename.txt").read()
contents[0:500]
```

> You might need `open("your-filename.txt", encoding="utf8").read()`

### Using regex

For some dumb reason you need to put `r` in front of the string you use when you're talking about regex. Just plain `"(\d\d\d)"` will usually work, but *sometimes* it won't and you'll need `r"(\d\d\d)`. It's best to just use the `r` all of the time, if you can remember!

### Using `.str.extract`

When you use `.str.extract`, you're always going to **capture one thing** and save it to a new column. You need to wrap the things you're interested in with parenthesis `(` `)`.

```python
df['phone_number'] = df['old_column'].str.extract(r"My phone number is (\d\d\d-\d\d\d-\d\d\d\d)")
```

### Setting pandas options

Pandas has a lot of options, like how many columns or rows it will show you, or how many characters it will show in a column before it stops showing you anything. Here are a few useful ones:

* `display.max_cols`: Number of columns to show at once
* `display.max_rows`: Number of rows to show at once
* `display.max_colwidth`: Maximum number of characters displayed from a string

You can set them using `pd.set_option("display.max_rows", 1000)`, for example, to show 1000 rows at a time. You can find a lot more at https://pandas.pydata.org/pandas-docs/stable/generated/pandas.set_option.html

### Regular expressions reference

I personally think http://www.regular-expressions.info/ is a wonderful wonderful reference (and tutorial), even if it's ugly! But here's a quick reference for you:

* `\d` is a digit
* `\d*` is zero or more digits 
* `\d+` is one or more digits
* `.` matches anything for ONE character
* `.*` is "give me anything forever"
* `\s` is whitespace, a.k.a. spaces and tabs
* `\w` is a word character, which includes capital and lowercase letters, numbers and hyphens.
* You can put `*` after anything, so `\w*` would mean "as many word characters as you can find"
* `\b` is a word boundary (you'll need the `r""` thing for this one)
* `( )` is a "capture group" for saving something
* `\1` is used when doing find/replace to say "put the first captured group here" (note, it's a dollar sign instead of a backslash in some editors)
* `[ABCDE]` is a character class, which means "match one of these, I don't care which"
* dollar sign means "end of the line"
* caret ^ means "beginning of the line"
* `\.` means "no really seriously I mean a period not just anything"
* You can use `\` with anything else that would normally be a special character, too, not just periods. `(` or `[` or whatever.

### Cleaning up extracted columns

Sometimes you get `\n` (newlines) or spaces or `\t` (tabs) or stuff at the beginning or the end of your column. `.str.strip()` will usually take care of that, just attach it after your `.str.extract()`

After you extract something, it's still a string even though you look at it and know it's a number. Use `.astype(int)` to turn it into an integer (no decimal) or `.astype(float)` to turn it into a float (yes decimal)

### Writing regular expressions in general

Even if I'm using regex in pandas or Python, I like to test them in my text editor with "Find." The highlighting really helps me see if I'm matching things! I also like to think "what stays the same?" when designing patterns, write those parts first, then fill in the blanks with what I want to capture.

## Importing

There might be more, I just wanted to put this up here for the `pd.set_option` part. It allows you to see a lot of content in a single column of pandas, which will be important for some parts below.

In [1]:
import re
import pandas as pd
pd.set_option('display.max_colwidth', 500)

# Part 1: Using `.str.extract` to pull data from columns in pandas

## 1.1 H&M

Open up `hm.csv` from the `scraped` directory. I want **four new columns**:

1. `price_original`, the original price, one of the new price
2. `price_discounted`, the discounted price
3. `pct_discount`, the percent discount
4. `article_id`, the article id (from the url)

Save as **hm_cleaned.csv**.

**Note:** When you look at it, it... won't look right. I don't know why, pandas is weird. Look at the `price` column by itself using `df['price']` before you write your regex.

**Tip:** Remember that `$` is a special regex symbol! You might need to escape it.

**Tip:** When doing `.str.extract`, the whole match doesn't get captured, only what you put `()` around! Think about anchoring to different points of the string, or things in the string.

**Tip:** Not all prices have cents!

**Tip:** Your first instinct about how to compute the percent discount is probably wrong

In [2]:
df = pd.read_csv('scraped/hm.csv')
df.head()

,name,price,url
0,Washed Linen Duvet Cover Set,$59.99 $129,http://www.hm.com/us/product/13472?article=13472-N
1,Candle in Glass Jar,$6.99 $17.99,http://www.hm.com/us/product/35079?article=35079-D
2,Glittery Cushion Cover,$7.99 $17.99,http://www.hm.com/us/product/72462?article=72462-A
3,Textured-weave Cushion Cover,$6.99 $12.99,http://www.hm.com/us/product/58926?article=58926-C
4,Stoneware Bowl,$17.99 $24.99,http://www.hm.com/us/product/74242?article=74242-A


In [3]:
df['price'].head()

0      $59.99 $129
1     $6.99 $17.99
2     $7.99 $17.99
3     $6.99 $12.99
4    $17.99 $24.99
Name: price, dtype: object

In [4]:
df['price_discounted'] = df['price'].str.extract("(\d+.\d+)").astype(float)
df['price_discounted'].head()

0    59.99
1     6.99
2     7.99
3     6.99
4    17.99
Name: price_discounted, dtype: float64

In [5]:
df['price_original'] = df['price'].str.extract("([\d.]+$)").astype(float)
df['price_original'].head()

0    129.00
1     17.99
2     17.99
3     12.99
4     24.99
Name: price_original, dtype: float64

In [6]:
df['pct_discount'] = round(100 - df['price_discounted'] / df['price_original'] * 100)
df['pct_discount'].head()

0    53.0
1    61.0
2    56.0
3    46.0
4    28.0
Name: pct_discount, dtype: float64

In [7]:
df['article_id'] = df['url'].str.extract("([\w\d-]+$)")
df['article_id'].head()

0    13472-N
1    35079-D
2    72462-A
3    58926-C
4    74242-A
Name: article_id, dtype: object

In [8]:
df.head()

,name,price,url,price_discounted,price_original,pct_discount,article_id
0,Washed Linen Duvet Cover Set,$59.99 $129,http://www.hm.com/us/product/13472?article=13472-N,59.99,129.00,53.0,13472-N
1,Candle in Glass Jar,$6.99 $17.99,http://www.hm.com/us/product/35079?article=35079-D,6.99,17.99,61.0,35079-D
2,Glittery Cushion Cover,$7.99 $17.99,http://www.hm.com/us/product/72462?article=72462-A,7.99,17.99,56.0,72462-A
3,Textured-weave Cushion Cover,$6.99 $12.99,http://www.hm.com/us/product/58926?article=58926-C,6.99,12.99,46.0,58926-C
4,Stoneware Bowl,$17.99 $24.99,http://www.hm.com/us/product/74242?article=74242-A,17.99,24.99,28.0,74242-A


In [9]:
df.to_csv('hm_cleaned.csv', index=False)

## 1.2 Sci-Fi Authors

Open up `sci-fi.csv` to clean. Get rid of the `\n` on the title and and give me six new columns:

* `avg_rating`
* `rating_count`
* `total_score`
* `score_votes`
* `series` the series the book belongs to
* `series_no` the book in the series that it is

For series, I'm talking about e.g. `(The Hunger Games, #1)` is `series` "The Hunter Games" and `series_no` 1.

Save as **sci-fi_cleaned.csv**.

**Tip:** You don't need regex to clean the title - there's a special thing that removes whitespace from the beginning/end of strings

**Tip:** Remember that `(` and `)` are special characters

**BONUS:** When you make the `total_score` column, pay close attention to it. If you notice the problem, fix it.

**BONUS:** You don't need these columns to be numbers, but life would be better if they were. 

In [10]:
df = pd.read_csv('scraped/sci-fi.csv')

In [11]:
df

,full_rating,full_score,rank,title,url
0,"4.07 avg rating — 785,502 ratings","\nscore: 28,539,\n and\n292 people voted\n \n \n",1,\nThe Handmaid's Tale\n,/book/show/38447.The_Handmaid_s_Tale
1,"4.34 avg rating — 5,212,935 ratings","\nscore: 27,566,\n and\n282 people voted\n \n \n",2,"\nThe Hunger Games (The Hunger Games, #1)\n",/book/show/2767052-the-hunger-games
2,"3.76 avg rating — 922,308 ratings","\nscore: 20,049,\n and\n205 people voted\n \n \n",3,"\nFrankenstein, or The Modern Prometheus\n",/book/show/18490.Frankenstein_or_The_Modern_Prometheus
3,"4.04 avg rating — 702,272 ratings","\nscore: 17,684,\n and\n185 people voted\n \n \n",4,"\nA Wrinkle in Time (A Wrinkle in Time Quintet, #1)\n",/book/show/18131.A_Wrinkle_in_Time
4,"4.06 avg rating — 77,664 ratings","\nscore: 16,070,\n and\n165 people voted\n \n \n",5,\nThe Left Hand of Darkness\n,/book/show/18423.The_Left_Hand_of_Darkness
5,"4.23 avg rating — 2,345,974 ratings","\nscore: 12,935,\n and\n134 people voted\n \n \n",6,"\nDivergent (Divergent, #1)\n",/book/show/13335037-divergent
6,"4.30 avg rating — 2,049,239 ratings","\nscore: 12,261,\n and\n128 people voted\n \n \n",7,"\nCatching Fire (The Hunger Games, #2)\n",/book/show/6148028-catching-fire
7,"4.12 avg rating — 1,379,452 ratings","\nscore: 11,238,\n and\n117 people voted\n \n \n",8,"\nThe Giver (The Giver, #1)\n",/book/show/3636.The_Giver
8,"4.19 avg rating — 57,605 ratings","\nscore: 10,246,\n and\n107 people voted\n \n \n",9,\nThe Dispossessed\n,/book/show/13651.The_Dispossessed
9,"4.20 avg rating — 53,473 ratings","\nscore: 9,907,\n and\n104 people voted\n \n \n",10,\nKindred\n,/book/show/60931.Kindred


In [12]:
df['title'] = df['title'].str.strip("\n")
df['title'].head()

0                                  The Handmaid's Tale
1              The Hunger Games (The Hunger Games, #1)
2               Frankenstein, or The Modern Prometheus
3    A Wrinkle in Time (A Wrinkle in Time Quintet, #1)
4                            The Left Hand of Darkness
Name: title, dtype: object

In [13]:
df['avg_rating'] = df.full_rating.str.extract("(\d.\d+)").astype(float)
df['avg_rating'].head()

0    4.07
1    4.34
2    3.76
3    4.04
4    4.06
Name: avg_rating, dtype: float64

In [14]:
df['rating_count'] = df.full_rating.str.extract("(\d[^.][\d,]+)")
df['rating_count'] = df['rating_count'].str.replace(",","").astype(float)
df['rating_count'].head()

0     785502.0
1    5212935.0
2     922308.0
3     702272.0
4      77664.0
Name: rating_count, dtype: float64

In [15]:
df['total_score'] = df.full_score.str.extract("([\d,]+\d)")
df['total_score'] = df['total_score'].str.replace(",","").astype(int)
df['total_score'].head()

0    28539
1    27566
2    20049
3    17684
4    16070
Name: total_score, dtype: int64

In [16]:
df['score_votes'] = df.full_score.str.extract("(\d+\s)").astype(int)
df['score_votes'].head()

0    292
1    282
2    205
3    185
4    165
Name: score_votes, dtype: int64

In [17]:
df['series'] = df.title.str.extract("[(](.*),")
df['series'].head()

0                          NaN
1             The Hunger Games
2                          NaN
3    A Wrinkle in Time Quintet
4                          NaN
Name: series, dtype: object

In [18]:
df['series_no'] = df.title.str.extract("#(\d-?\d?)[)]")
df['series_no'].head()

0    NaN
1      1
2    NaN
3      1
4    NaN
Name: series_no, dtype: object

In [19]:
df.head()

,full_rating,full_score,rank,title,url,avg_rating,rating_count,total_score,score_votes,series,series_no
0,"4.07 avg rating — 785,502 ratings","\nscore: 28,539,\n and\n292 people voted\n \n \n",1,The Handmaid's Tale,/book/show/38447.The_Handmaid_s_Tale,4.07,785502.0,28539,292,NaN,NaN
1,"4.34 avg rating — 5,212,935 ratings","\nscore: 27,566,\n and\n282 people voted\n \n \n",2,"The Hunger Games (The Hunger Games, #1)",/book/show/2767052-the-hunger-games,4.34,5212935.0,27566,282,The Hunger Games,1
2,"3.76 avg rating — 922,308 ratings","\nscore: 20,049,\n and\n205 people voted\n \n \n",3,"Frankenstein, or The Modern Prometheus",/book/show/18490.Frankenstein_or_The_Modern_Prometheus,3.76,922308.0,20049,205,NaN,NaN
3,"4.04 avg rating — 702,272 ratings","\nscore: 17,684,\n and\n185 people voted\n \n \n",4,"A Wrinkle in Time (A Wrinkle in Time Quintet, #1)",/book/show/18131.A_Wrinkle_in_Time,4.04,702272.0,17684,185,A Wrinkle in Time Quintet,1
4,"4.06 avg rating — 77,664 ratings","\nscore: 16,070,\n and\n165 people voted\n \n \n",5,The Left Hand of Darkness,/book/show/18423.The_Left_Hand_of_Darkness,4.06,77664.0,16070,165,NaN,NaN


In [20]:
df.to_csv('sci-fi_cleaned.csv', index=False)

## 1.3 Where you're just doing one of my former students' projects

Once upon a time my student Stefan did a project that involved some lawyer stuff. Most of the content was in PDFs, though! I converted them to text files and put them into the `pdfs` folder, and gave you code below to open up each of them and save their contents into a dataframe.

What a nice dataframe! I want you to add the following columns to it:

* `lawyer_app`, the applicant's lawyer (pro se means that they did it themselves, that's fine)
* `lawyer_gov`, the government's lawyer
* `judge`, the name of the judge
* `access`, whether the clearance is granted or denied (although you might miss a few)

Save as **court_cleaned.csv**.

**Note:** You can look at the original PDFs, they're also included.

**Note:** This uses a fun utility called `glob`, which is mostly fun because you use it as `glob.glob`. It's used to find files that match a certain filename pattern.

**BONUS:** You'll be happy once you get the judge, but make sure it doesn't have any extra punctuation on it.

**BONUS:** You can for some words using `.str.contains("blah")` and save it into new columns. Maybe `has_debt`, `has_bankruptcy`, etc.

> It's okay if it isn't perfect. Converting PDF into data rarely is! Usually you get 90% of it done with computers, then send people to enter the other 10% by hand.

In [21]:
import glob
filenames = glob.glob("pdfs/*.txt")
contents = [open(filename, encoding="utf8").read() for filename in filenames]
df = pd.DataFrame({'filename': filenames, 'content': contents})

In [22]:
df.head()

,filename,content
0,pdfs/11-11916.h1.pdf.txt,"\n\nDEPARTMENT OF DEFENSE \n\nDEFENSE OFFICE OF HEARINGS AND APPEALS \n\n \n \n\n \n \nIn the matter of: \n \n \n \nApplicant for Security Clearance \n\nREDACTED \n\n \n\nISCR Case No. 11-11916 \n\nMENDEZ, Francisco, Administrative Judge: \n\n \nApplicant did not mitigate security concerns raised by his exercise of foreign \ncitizenship, including the possession of a current foreign passpor..."
1,pdfs/12-01601.a1.pdf.txt,"KEYWORD: Guideline F\n\nDIGEST: Most of the document Applicant submitted on appeal were not previously submitted to\nthe Judge and the Board is prohibited from receiving or considering them. Adverse decision\naffirmed.\n\nCASENO: 12-01601.a1\n\nDATE: 09/23/2016\n\nIn Re:\n\n-------\n\nApplicant for Security Clearance\n\nDATE: September 23, 2016\n\nISCR Case No. 12-01601\n\n)\n)\n)\n)\n)\n)\n)\n)\n\nAPPEAL BOARD DECISION\n\nAPPEARANCES\n\nFOR GOVERNMENT\n\nJames B. Norman, Esq., Chief Depart..."
2,pdfs/11-03073.h1.pdf.txt,"\n\n DEPARTMENT OF DEFENSE \n\n DEFENSE OFFICE OF HEARINGS AND APPEALS \n\n \n \nIn the matter of: \n \n \n \nApplicant for Security Clearance \n\n \n\n \n\nISCR Case No. 11-03073 \n\n \n \n\n) \n) \n) \n) \n) \n \n \n\n \n \n\nAppearances \n\n______________ \n\n \nDecision \n\n______________ \n\n \n\n \n\n \n\n \n \n\nFor Government: Robert J. Kilmartin, Esq., Department Counse..."
3,pdfs/11-04909.h1.pdf.txt,"\n\n DEFENSE OFFICE OF HEARINGS AND APPEALS \n\n DEPARTMENT OF DEFENSE \n\n \n \n\n \n\n \nIn the matter of: \n \n \n \nApplicant for Security Clearance \n\n \n \n\n \n\nISCR Case No. 11-04909 \n\n \n\nFor Government: Richard Stevens, Esq., Department Counsel \n\nFor Applicant: Pro se \n\n \n\n \nDUFFY, James F., Administrative Judge: \n\n \nApplicant mitigated \n\nconsiderations). Clearance is ..."
4,pdfs/11-08313.h1.pdf.txt,"\n\n DEPARTMENT OF DEFENSE \n DEFENSE OFFICE OF HEARINGS AND APPEALS \n\n \n\nISCR Case No. 11-08313 \n\n \nIn the matter of: \n \n \n \nApplicant for Security Clearance \n\n--------------- \n \n\n \n\n \n \n\n) \n) \n) \n) \n) \n \n\n \n \n\nAppearances \n\n______________ \n\n \nDecision \n\n______________ \n\nFor Government: Julie R. Mendez, Esquire, Department Counsel \n\nFo..."


Okay, now do the work and **make those new columns!**

In [23]:
df['lawyer_gov'] = df.content.str.extract("For Government: ([\w\s.]+)")
df['lawyer_gov']

0          David F. Hayes
1                     NaN
2     Robert J. Kilmartin
3         Richard Stevens
4         Julie R. Mendez
5          David F. Hayes
6       Stephanie C. Hess
7            Tara Karoian
8         Richard Stevens
9        Robert Kilmartin
10         Eric Borgstrom
11                    NaN
12          Tovah Minster
13        Julie R. Mendez
14          Erin Thompson
Name: lawyer_gov, dtype: object

In [24]:
#df['lawyer_app'] = 
df.content.str.extract("For Applicant: ([\w\s.]+[^\n,])")

,0
0,Pro se \n\nAppearances \n\n______________ \n\n \nDecision \n\n______________ \n\n \n\n \n \n\n)
1,NaN
2,Mark S. Zaid
3,Pro se \n\n \n\n \nDUFFY
4,Pro se \n\n \n \n\n \n\nMARSHALL
5,Pro se \n\n \n \n\n______________ \n\n \n\nDecision \n\n______________ \n\n \n\n \n\n \n \n\nHOWE
6,Pro se \n\nAppearances \n\n______________ \n\n \nDecision \n\n______________ \n\n \n\n \n\n \n\n \n \n\n \n\nCOACHER
7,Pro se \n\n \n\n \n\n)
8,Ryan C. Nerney
9,Pro Se \n\n \n\nApril 28


In [25]:
df['judge'] = df.content.str.extract("\n([A-Z]+,[\w\s.]+), Administrative Judge:")
df['judge']

0          MENDEZ, Francisco
1                        NaN
2        LOUGHRAN, Edward W.
3            DUFFY, James F.
4                        NaN
5            HOWE, Philip S.
6         COACHER, Robert E.
7            DUFFY, James F.
8     GALES, Robert Robinson
9     GOLDSTEIN, Jennifer I.
10            HOGAN, Erin C.
11                       NaN
12          MOGUL, Martin H.
13              HARVEY, Mark
14                       NaN
Name: judge, dtype: object

In [26]:
df['access'] = df.content.str.extract("access to classified information is (\w+)")
df['access']

0         NaN
1         NaN
2     granted
3         NaN
4      denied
5      denied
6      denied
7      denied
8     granted
9      denied
10    granted
11        NaN
12     denied
13    granted
14    granted
Name: access, dtype: object

# Reading books

When you're doing text work, you're legally obligated work on Jane Austen's Pride and Prejudice (at least I *think* so). Let's do some naive analysis of it!

## Read in Jane Austen's Pride and Prejudice (without moving the file!)

It's in the `data/` directory, and named `Austen_Pride.txt`.

In [27]:
f = open('data/Austen_Pride.txt')
text = f.read()

## Look at the first 500 or so characters of it 

In [28]:
text[:500]

' Pride and Prejudice\nby Jane Austen\nChapter 1\nIt is a truth universally acknowledged, that a single man in possession of a good fortune, must be in want of a wife.\nHowever little known the feelings or views of such a man may be on his first entering a neighbourhood, this truth is so well fixed in the minds of the surrounding families, that he is considered the rightful property of some one or other of their daughters.\n"My dear Mr. Bennet," said his lady to him one day, "have you heard that Nethe'

## Use a regular expression to find every "he" or "she" in the book. There should be about 3000 of them.

**Tip:** Do you know about **word boundaries?** `\b` means "the beginning of end of a word."

**Tip:** You might also want to use `re.IGNORECASE`. Maybe you'll need to google it? 

**Tip:** Do NOT use `re.compile`

In [29]:
pronouns = re.findall(r"\bs*he\b", text, re.IGNORECASE)
len(pronouns)

3047

## Use a regular expression to find those same "he" or "she"s, but also match *the word after it*

The first four should be:

* he is
* he had
* she told
* he came

In [30]:
match = re.findall(r"(\bs*he\s\w+\b)", text, re.IGNORECASE)
match[:4]

['he is', 'he had', 'she told', 'he came']

## Use capture groups to save the pronoun (he/she) as one match and the word as another

The first five should look like

```
[('he', 'is'),
 ('he', 'had'),
 ('she', 'told'),
 ('he', 'came'),
 ('he', 'agreed')]```

In [31]:
match = re.findall(r"(\bs*he\b) (\b\w+\b)", text, re.IGNORECASE)
match[:5]

[('he', 'is'),
 ('he', 'had'),
 ('she', 'told'),
 ('he', 'came'),
 ('he', 'agreed')]

## Save those matches into a dataframe

You can give the column names with `columns=['pronoun', 'verb']`

In [32]:
import pandas as pd
df = pd.DataFrame(match, columns=['pronoun', 'verb'])
df.head()

,pronoun,verb
0,he,is
1,he,had
2,she,told
3,he,came
4,he,agreed


## How many times is each pronoun used?

In [33]:
df.pronoun.value_counts()

she    1322
he     1054
She     325
He      234
Name: pronoun, dtype: int64

## Oh, wait, clean that up.

Make it only 'he' and 'she' lowercase.

It should be about 1600 'she' and 1300 'he'

In [34]:
df.pronoun.str.lower().value_counts()

she    1647
he     1288
Name: pronoun, dtype: int64

## What are the top 20 most common verbs?

In [35]:
df.verb.value_counts().head(20)

was          372
had          371
could        172
is           139
would         94
has           70
did           67
will          50
might         46
should        41
felt          38
must          37
said          33
saw           32
added         31
thought       31
then          26
replied       22
looked        21
continued     21
Name: verb, dtype: int64

## What are the top 20 most common verbs for 'he', and the top 20 most common for 'she'

**Tip:** Don't use groupby, just filter. If you want to know how, though, you can also look at "value counts for different categories" on [this page](http://jonathansoma.com/lede/foundations-2017/classes/more-pandas/class-notes/)

In [36]:
df[df.pronoun.str.lower() == 'he'].verb.value_counts().head(20)

had        166
was        160
is          74
has         49
could       40
would       35
did         29
should      26
must        24
will        24
might       21
replied     14
may         13
thought     13
came        13
said        13
never       12
does        12
then        11
meant       11
Name: verb, dtype: int64

In [37]:
df[df.pronoun.str.lower() == 'she'].verb.value_counts().head(20)

was          212
had          205
could        132
is            65
would         59
did           38
felt          33
saw           29
will          26
might         25
added         23
has           21
said          20
thought       18
should        15
then          15
found         13
must          13
soon          12
continued     11
Name: verb, dtype: int64

## Who cries more, men or women? Give me a percentage answer.

**Tip:** It's `cried`, because of, you know, how books are written

In [38]:
df[df.verb == 'cried'].pronoun.value_counts(normalize=True) * 100

she    91.666667
he      8.333333
Name: pronoun, dtype: float64

## How much more common is 'he' than 'she' in J.R.R. Tolkein's Fellowship of the Ring? How does that compare to Pride and Prejudice?

The book is in the same directory.

In [39]:
f = open('data/Lord of the Rings - 01 - The Fellowship of the Ring - J. R. R. Tolkien - 1955.txt')
text = f.read()

In [40]:
pronouns = re.findall(r"\bs*he\b", text, re.IGNORECASE)
df = pd.DataFrame(pronouns, columns=['pronoun'])
df.pronoun.str.lower().value_counts()

he     3062
she     159
Name: pronoun, dtype: int64